In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! ls 'drive/My Drive/프로젝트/nlp'

data_check.ipynb  train_filtered.csv


In [ ]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

125

In [ ]:
# hugging face install
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.2 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 42.3 MB/s 
     |████████████████████████████████| 3.3 MB 39.4 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Install helper functions.
!pip install -q git+https://github.com/gmihaila/ml_things.git

     |████████████████████████████████| 64 kB 1.8 MB/s 
     |████████████████████████████████| 11.2 MB 8.6 MB/s 
     |████████████████████████████████| 873 kB 49.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import io
import os
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification,
                          GPT2LMHeadModel)

In [ ]:
import tqdm 

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd

import shutil

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Running on device: {device}')

Running on device: cuda:0


In [ ]:
path = 'drive/My Drive/프로젝트/nlp/train_filtered.csv'

In [ ]:
data = pd.read_csv(path)

In [ ]:
data

,Artist,Song,Genre,Language,Lyrics
0,12 stones,world so cold,Rock,en,"It starts with pain, followed by hate\nFueled ..."
1,12 stones,broken,Rock,en,Freedom!\nAlone again again alone\nPatiently w...
2,12 stones,3 leaf loser,Rock,en,"Biting the hand that feeds you, lying to the v..."
3,12 stones,anthem for the underdog,Rock,en,You say you know just who I am\nBut you can't ...
4,12 stones,adrenaline,Rock,en,My heart is beating faster can't control these...
...,...,...,...,...,...
249943,bobby womack,i wish he didn t trust me so much,R&B,en,I'm the best friend he's got I'd give him the ...
249944,bad boys blue,i totally miss you,Pop,en,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
249945,celine dion,sorry for love,Pop,en,Forgive me for the things That I never said to...
249946,dan bern,cure for aids,Indie,en,The day they found a cure for AIDS The day the...


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
## data split
## using 200,000 takes too much time
## use 1/10


In [ ]:

set_seed(123)
epochs = 2
batch_size = 32

# Pad or truncate text sequences to a specific length
max_length = 200

model_name_or_path = 'gpt2'

# Dictionary of labels and their id - this will be used to convert.
# String labels to number ids.
labels_ids = {'Rock': 0, 'Pop': 1, 'Metal': 2, 'Jazz': 3, 'Folk': 4, 'Indie': 5, 'R&B': 6, 'Hip-Hop': 7, 'Electronic': 8, 'Country': 9}
n_labels = len(labels_ids)


In [ ]:
class Gpt2ClassificationCollator(object):

    def __init__(self, use_tokenizer, labels_encoder, max_sequence_len=None):
        self.use_tokenizer = use_tokenizer
        self.max_sequence_len = use_tokenizer.model_max_length if max_sequence_len is None else max_sequence_len
        self.labels_encoder = labels_encoder

        return

    def __call__(self, sequences):
        texts = [sequence['text'] for sequence in sequences]
        labels = [sequence['label'] for sequence in sequences]
        labels = [self.labels_encoder[label] for label in labels]
        inputs = self.use_tokenizer(text=texts, return_tensors="pt", padding=True, truncation=True,  max_length=self.max_sequence_len)
        inputs.update({'labels':torch.tensor(labels)})

        return inputs


def train(dataloader, optimizer_, scheduler_, device_):
  global model

  predictions_labels = []
  true_labels = []
  total_loss = 0

  model.train()

  for batch in dataloader:
    true_labels += batch['labels'].numpy().flatten().tolist()
    
    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}
    model.zero_grad()
    outputs = model(**batch)
    loss, logits = outputs[:2]
    total_loss += loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer_.step()
    scheduler_.step()
    logits = logits.detach().cpu().numpy()
    predictions_labels += logits.argmax(axis=-1).flatten().tolist()

  avg_epoch_loss = total_loss / len(dataloader)
  
  return true_labels, predictions_labels, avg_epoch_loss



def validation(dataloader, device_):
  global model
  predictions_labels = []
  true_labels = []
  total_loss = 0

  model.eval()
  for batch in dataloader:

    true_labels += batch['labels'].numpy().flatten().tolist()

    batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

    with torch.no_grad():        
        outputs = model(**batch)
        loss, logits = outputs[:2]
        logits = logits.detach().cpu().numpy()
        total_loss += loss.item()
        predict_content = logits.argmax(axis=-1).flatten().tolist()

        # update list
        predictions_labels += predict_content

  # Calculate the average loss over the training data.
  avg_epoch_loss = total_loss / len(dataloader)

  return true_labels, predictions_labels, avg_epoch_loss

In [ ]:
# Get model configuration.
print('Loading configuraiton...')
model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=n_labels)

# Get model's tokenizer.
print('Loading tokenizer...')
tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
# default to left padding
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token


# Get the actual model.
print('Loading model...')
model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path, config=model_config)

# resize model embedding to match new tokenizer
model.resize_token_embeddings(len(tokenizer))

# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)

Loading configuraiton...
Loading tokenizer...
Loading model...


Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded to `cuda:0`


In [ ]:
## datset

### Prepare data
#data = pd.read_csv(path)
# Artist	Song	Genre	Language	Lyrics

# data_drop : Genre Lyrics
data_drop = data.drop(['Artist', 'Song', 'Language'], axis=1)

#Drop the songs with lyrics too long (after more than 1024 tokens, does not work)
#df = df[df['Lyric'].apply(lambda x: len(x.split(' ')) < 350)]


In [ ]:
# 중복 데이터 제거
data_drop.drop_duplicates('Lyrics')


,Genre,Lyrics
0,Rock,"It starts with pain, followed by hate\nFueled ..."
1,Rock,Freedom!\nAlone again again alone\nPatiently w...
2,Rock,"Biting the hand that feeds you, lying to the v..."
3,Rock,You say you know just who I am\nBut you can't ...
4,Rock,My heart is beating faster can't control these...
...,...,...
249943,R&B,I'm the best friend he's got I'd give him the ...
249944,Pop,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
249945,Pop,Forgive me for the things That I never said to...
249946,Indie,The day they found a cure for AIDS The day the...


In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_drop, test_size=0.3, random_state=123)


In [ ]:
train_data

,Genre,Lyrics
34449,Pop,Give me attention\nI need it now\nToo much dis...
172104,Rock,"A car's got gasoline to run, down the road\nA ..."
4730,Pop,celebration\npineapples and kumquats\nsimultan...
172835,Pop,Last year's troubles are so old fashioned\nThe...
161013,Pop,You'd probably think I was psychotic (if you k...
...,...,...
192476,Rock,What do you think they would say\nIf I stood u...
17730,Rock,This is my time with you\nI'm not giving it ba...
28030,Pop,"Come away, little loss\nCome away to the water..."
15725,Rock,You pull me down\nTo the floor\nOf our apartme...


In [ ]:
test_data

,Genre,Lyrics
206403,Indie,You only like him because he's sexually appeal...
93351,Pop,Living la dolce vita\nLife couldn't get much s...
233569,Rock,"Another lonely night, on the telephone we talk..."
210351,Rock,"Ah, ah ah, ah ah, ah!\nAh, ah ah, ah ah, ah!\n..."
81488,Rock,When I speak to the flowers in my backyard\nWh...
...,...,...
31969,Rock,Get up early every morning\nJust to put your m...
247169,Country,If you never come to me again I've had the lov...
99975,Pop,"She didn't have a reason to go, oh no\nShe did..."
227338,Rock,they were so ecstatic when the letter arrived\...


In [ ]:
# make dataset
class SongLyrics(Dataset):  
    def __init__(self, data, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.texts = []
        self.labels = []

        for row in data['Lyrics']:
          #print(row)
          self.texts.append(row)
        
        for gen in data['Genre']:
          #print(data_drop[data_drop['Lyrics']==row]['Genre'])
          self.labels.append(gen)

        self.lyrics_count = len(self.labels)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        #return self.lyrics[item]
        return {'text':self.texts[item],
                'label':self.labels[item]}
    

In [ ]:
train_dataset = SongLyrics(train_data, truncate=True, gpt2_type="gpt2")
test_dataset = SongLyrics(test_data, truncate=True, gpt2_type="gpt2")

In [ ]:
train_dataset[0]

{'label': 'Pop',
 'text': "Give me attention\nI need it now\nToo much distance\nTo measure it out (out loud)\nTracing patterns\nAcross a personal map\nAnd making pictures\nWhere the lines overlap\nWhere the lines overlap\nNo one is as lucky as us\nWe're not at the end but\nOh we already won\nOh no, no one\nIs as lucky as us\nIs as lucky as us\nCall me over\nAnd tell me how\nYou got so far\nNever making a single sound\nI'm not used to it but I can learn\nThere's nothing to it\nI've never been happier\nI've never been happier\nNo one is as lucky as us\nWe're not at the end but\nOh we already won\nOh no, no one\nIs as lucky as us\nIs as lucky as us\nNow I've got a feeling if I sang this loud enough\nYou would sing it back to me\nNow I've got a feeling if I sang this loud enough\nYou would sing it back to me\nI've got a feeling\nI've got a feeling\nYou, will sing it back to me\nYou, will sing it back to me\nNo one is as lucky as us\nWe're not at the end but\nOh we already won\nNo one\nNo o

In [ ]:
gpt2_classificaiton_collator = Gpt2ClassificationCollator(use_tokenizer=tokenizer, 
                                                          labels_encoder=labels_ids, 
                                                          max_sequence_len=max_length)


print('Dealing with Train...')
# Create pytorch dataset.
# 이미 pandas로 dataset을 받아왔으니까 괜찮지 않을까...?
# train_dataset을 새로 정의해야함..


# Move pytorch dataset into dataloader.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=gpt2_classificaiton_collator)
print('Created `train_dataloader` with %d batches!'%len(train_dataloader))

print()


print('Dealing with Test...')
# Create pytorch dataset.


# Move pytorch dataset into dataloader.
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=gpt2_classificaiton_collator)
print('Created `eval_dataloader` with %d batches!'%len(test_dataloader))


Dealing with Train...
Created `train_dataloader` with 5468 batches!

Dealing with Test...
Created `eval_dataloader` with 2344 batches!


In [ ]:
train_dataloader

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                  )


total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

all_loss = {'train_loss':[], 'test_loss':[]}
all_acc = {'train_acc':[], 'val_acc':[]}

## train
for epoch in range(epochs):
  print()
  print('Training on batches : {}epoch').format(epoch)
  train_labels, train_predict, train_loss = train(train_dataloader, optimizer, scheduler, device)
  train_acc = accuracy_score(train_labels, train_predict)

  # Get prediction form model on validation data. 
  print('Test results')
  test_labels, test_predict, test_loss = validation(test_dataloader, device)
  val_acc = accuracy_score(test_labels, test_predict)

  # Print loss and accuracy values to see how training evolves.
  print("  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(train_loss, test_loss, train_acc, val_acc))
  print()

  # Store the loss value for plotting the learning curve.
  all_loss['train_loss'].append(train_loss)
  all_loss['test_loss'].append(test_loss)
  all_acc['train_acc'].append(train_acc)
  all_acc['val_acc'].append(val_acc)



Epoch

Training on batches...
Test on batches...
  train_loss: 1.17733 - val_loss: 1.08713 - train_acc: 0.55323 - valid_acc: 0.59569


Training on batches...
Test on batches...
  train_loss: 1.05260 - val_loss: 1.05961 - train_acc: 0.60632 - valid_acc: 0.60661



NameError: ignored

In [ ]:
## print final f1-score
true_labels, predictions_labels, avg_epoch_loss = validation(test_dataloader, device)

# Create and print the evaluation report.
evaluation_report = classification_report(true_labels, predictions_labels, labels=list(labels_ids.values()), target_names=list(labels_ids.keys()))
print(evaluation_report)

              precision    recall  f1-score   support

        Rock       0.59      0.78      0.67     32093
         Pop       0.67      0.58      0.62     25860
       Metal       0.56      0.44      0.50      5657
        Jazz       0.60      0.51      0.55      4022
        Folk       0.37      0.03      0.05      2496
       Indie       0.41      0.00      0.01      2183
         R&B       0.37      0.05      0.08       846
     Hip-Hop       0.65      0.73      0.69       661
  Electronic       0.29      0.04      0.07       589
     Country       0.33      0.52      0.40       578

    accuracy                           0.61     74985
   macro avg       0.48      0.37      0.36     74985
weighted avg       0.60      0.61      0.58     74985



# **Check only 30 songs**

In [ ]:
a_data, b_data = train_test_split(test_data, test_size=0.0005, random_state=123)

In [ ]:
b_data

,Genre,Lyrics
208906,Rock,Life has been unfaithful\nAnd it all promised ...
125507,Pop,Tell me what happened this time\nMade you come...
138399,Metal,Night has fallen\nAnd the darkness hides an ev...
25663,Rock,Why we're always headed for disaster\nWas I on...
136387,Rock,Walking down an empty road\nI saw a man in old...
73221,Pop,[Kokane]\nIn this times\nOf hate and pain\nWe ...
120761,Pop,This has been no walk in the park\nI feel like...
32439,Pop,"You, you like driving on a Sunday\nYou, you li..."
80782,Pop,"Loving you, I thought I couldn't get no higher..."
110417,Rock,I hear the drums echoing tonight\nBut she hear...


In [ ]:
test_dataset30 = SongLyrics(b_data, truncate=True, gpt2_type="gpt2")

In [ ]:
test_dataset30

In [ ]:

print('Dealing with Train...')
# Create pytorch dataset.
# 이미 pandas로 dataset을 받아왔으니까 괜찮지 않을까...?
# train_dataset을 새로 정의해야함..

print('Dealing with Test...')
# Create pytorch dataset.
# Move pytorch dataset into dataloader.
test_dataloader30 = DataLoader(test_dataset30, batch_size=16, shuffle=False, collate_fn=gpt2_classificaiton_collator)
print('Created `eval_dataloader` with %d batches!'%len(test_dataloader30))


Dealing with Train...
Dealing with Test...
Created `eval_dataloader` with 3 batches!


In [ ]:
true_labels, predictions_labels, avg_epoch_loss = validation(test_dataloader30, device)
val_acc = accuracy_score(test_labels, test_predict)

# Print loss and accuracy values to see how training evolves.
print("  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(train_loss, test_loss, train_acc, val_acc))

  train_loss: 1.05260 - val_loss: 1.05961 - train_acc: 0.60632 - valid_acc: 0.60661


In [ ]:
print(true_labels)
print(predictions_labels)

[0, 1, 2, 0, 0, 1, 1, 1, 1, 0, 0, 0, 2, 0, 1, 4, 5, 1, 1, 0, 0, 1, 5, 1, 0, 2, 0, 0, 0, 1, 1, 2, 0, 2, 0, 1, 0, 0]
[0, 0, 2, 1, 0, 1, 1, 1, 1, 0, 0, 0, 2, 0, 3, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 2, 1, 0, 1, 1, 1, 0, 0, 2, 0, 1, 1, 0]
